In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

df = pd.concat([train_df, test_df])

df.head(25)

C:\Users\Me\AppData\Local\Temp\ipykernel_14752\2582827541.py:7: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('data/train.csv')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.843333,3,...,Good,809.98,23.933795,NaN,No,49.574949,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,34847.84,3037.986667,2,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good


In [2]:
df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

**1. Data Cleaning and Preprocessing:**
So many missing values, we will need to treat each column depending on the nature of the data in each column and the overall context of your project.

In [3]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Name                       15000
Monthly_Inhand_Salary      22500
Type_of_Loan               17112
Num_of_Delayed_Payment     10500
Num_Credit_Inquiries        3000
Credit_History_Age         13500
Amount_invested_monthly     6750
Monthly_Balance             1762
Credit_Score               50000
dtype: int64

In [4]:
columns_to_drop = ['ID','SSN']
df = df.drop(columns=columns_to_drop) #<--- dropping columns that are not needed
df.columns

Index(['Customer_ID', 'Month', 'Name', 'Age', 'Occupation', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [5]:
df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
Index: 150000 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Customer_ID              150000 non-null  object
 1   Month                    150000 non-null  object
 2   Name                     135000 non-null  object
 3   Age                      150000 non-null  object
 4   Occupation               150000 non-null  object
 5   Annual_Income            150000 non-null  object
 6   Num_of_Loan              150000 non-null  object
 7   Type_of_Loan             132888 non-null  object
 8   Num_of_Delayed_Payment   139500 non-null  object
 9   Changed_Credit_Limit     150000 non-null  object
 10  Credit_Mix               150000 non-null  object
 11  Outstanding_Debt         150000 non-null  object
 12  Credit_History_Age       136500 non-null  object
 13  Payment_of_Min_Amount    150000 non-null  object
 14  Amount_invested_monthly  1

In [6]:
df['Customer_ID']             = df.Customer_ID.apply(lambda x: int(x[4:], 16))
df['Month']                   = pd.to_datetime(df.Month, format='%B').dt.month
df['Age']                     = df['Age'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['Annual_Income']           = df['Annual_Income'].str.replace(r'\D', '', regex=True).astype(float)
df['Num_of_Loan']             = df.Num_of_Loan.astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['Num_of_Delayed_Payment']  = df['Num_of_Delayed_Payment'].str.replace(r'\D', '', regex=True).astype(float)
df['Num_Credit_Inquiries']    = df['Num_Credit_Inquiries'].astype(str).str.replace(r'\D', '', regex=True)
df['Num_Credit_Inquiries']    = df['Num_Credit_Inquiries'].replace('', np.nan).astype(float)
df['Changed_Credit_Limit']    = df['Changed_Credit_Limit'].str.replace(r'_', '0').astype(float)
df['Outstanding_Debt']        = df['Outstanding_Debt'].str.replace(r'(\d)_', r'\1', regex=True).astype(float)
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].replace('__10000__', np.nan).astype(float)
df['Monthly_Balance']         = df['Monthly_Balance'].replace('__-333333333333333333333333333__', np.nan).astype(float)

In [7]:
df_check = df.copy()
df_check.shape

(150000, 26)

In [8]:
def text_cleaning(data):
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip('_ ,"')

In [9]:
df = df_check.applymap(text_cleaning).replace(['', 'nan', '!@9#%8', '#F%$D@*&8', 'NaN'], np.NaN)

C:\Users\Me\AppData\Local\Temp\ipykernel_14752\1035433873.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df_check.applymap(text_cleaning).replace(['', 'nan', '!@9#%8', '#F%$D@*&8', 'NaN'], np.NaN)


In [10]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Name                       15000
Occupation                 10500
Monthly_Inhand_Salary      22500
Type_of_Loan               17112
Num_of_Delayed_Payment     10500
Num_Credit_Inquiries        3000
Credit_Mix                 30000
Credit_History_Age         13500
Amount_invested_monthly    13230
Payment_Behaviour          11400
Monthly_Balance             1777
Credit_Score               50000
dtype: int64

In [11]:
import scipy.stats as stats
import numpy as np
import pandas as pd

def FillMissingWithGroupMode(df, group_column, target_column):
    # Function to convert None to NaN and fill NaN with the mode of the group
    def fill_mode_per_group(data, group, column):
        # Replace None with NaN
        data[column] = data[column].replace([None], np.nan)
        # Calculate and fill the mode for each group
        filled_data = data.groupby(group)[column].transform(lambda x: x.fillna(x.mode().iloc[0]))
        return filled_data

    # Display before filling NaN
    print(f'\nBefore filling NaN in {target_column}:')
    print(df[target_column].isna().sum(), "missing values")
    print(df.groupby(group_column)[target_column].apply(list).head())

    # Fill NaN values
    df[target_column] = fill_mode_per_group(df, group_column, target_column)

    # Display after filling NaN
    print(f'\nAfter filling NaN in {target_column}:')
    print(df[target_column].isna().sum(), "missing values")
    print(df.groupby(group_column)[target_column].apply(list).head())


In [12]:
#<--- Name
FillMissingWithGroupMode(df, 'Customer_ID', 'Name')


Before filling NaN in Name:
15000 missing values
Customer_ID
1006    [Matthias Blamontb, Matthias Blamontb, Matthia...
1007    [nan, Soyoung Kimu, Soyoung Kimu, Soyoung Kimu...
1008    [Koht, Koht, Koht, Koht, Koht, Koht, Koht, nan...
1009    [Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, Edd, ...
1011    [Terry Wadeu, Terry Wadeu, Terry Wadeu, Terry ...
Name: Name, dtype: object


In [ ]:
#<--- Name
FillMissingWithGroupMode(df, 'Customer_ID', 'Payment_Behaviour')

In [ ]:
#<--- Credit_Mix
FillMissingWithGroupMode(df, 'Customer_ID', 'Credit_Mix')

In [ ]:
#<--- Occupation
FillMissingWithGroupMode(df, 'Customer_ID', 'Occupation')

In [ ]:
#<--- Type_of_Loan
df['Type_of_Loan'] = df['Type_of_Loan'].apply(lambda x: x.lower().replace('and ', '').replace(', ', ',').strip() if pd.notna(x) else x)
import re
def get_Diff_Values_Colum(df_column, diff_value=[], sep=',', replace=''):
    column = df_column.dropna()
    for i in column:
        if sep not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), re.sub(replace, '', i).split(sep)):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)
df['Type_of_Loan'].replace([np.NaN], 'No Data', inplace=True)
get_Diff_Values_Colum(df['Type_of_Loan'])

In [ ]:
#<---Num_of_Delayed_Payment
percentile_95 = df['Num_of_Delayed_Payment'].quantile(0.95)
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].apply(lambda x: percentile_95 if x > percentile_95 else x)

df['Num_of_Delayed_Payment'] = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.fillna(x.median()))

overall_median = df['Num_of_Delayed_Payment'].median()
df['Num_of_Delayed_Payment'].fillna(overall_median, inplace=True)


In [ ]:
#<---Num_Credit_Inquiries
percentile_95_inquiries = df['Num_Credit_Inquiries'].quantile(0.95)
df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].apply(lambda x: percentile_95_inquiries if x > percentile_95_inquiries else x)

df['Num_Credit_Inquiries'] = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.fillna(x.median()))

overall_median_inquiries = df['Num_Credit_Inquiries'].median()
df['Num_Credit_Inquiries'].fillna(overall_median_inquiries, inplace=True)

In [ ]:
#<---Credit_History_Age
def convert_to_total_months(age_str):
    if pd.isna(age_str):
        return None
    parts = age_str.split(' ')
    years = int(parts[0]) if parts[0].isdigit() else 0
    months = int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else 0
    return years * 12 + months

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert_to_total_months)

df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda x: x.fillna(x.median()))

overall_median_credit_history = df['Credit_History_Age'].median()
df['Credit_History_Age'].fillna(overall_median_credit_history, inplace=True)

In [ ]:
# Group by 'Customer_ID' and fill NaNs with the median per customer
df['Amount_invested_monthly'] = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.fillna(x.median()))

# In case the entire 'Amount_invested_monthly' for a customer group is NaN, fill with overall median
overall_median_investment = df['Amount_invested_monthly'].median()
df['Amount_invested_monthly'].fillna(overall_median_investment, inplace=True)

In [ ]:
#<---Monthly_Balance
df['Monthly_Balance'] = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.fillna(x.median()))

overall_median_balance = df['Monthly_Balance'].median()
df['Monthly_Balance'].fillna(overall_median_balance, inplace=True)


In [ ]:
#<--- Monthly_Inhand_Salary (Each customer had stable income in dataset)
FillMissingWithGroupMode(df, 'Customer_ID', 'Monthly_Inhand_Salary')


In [ ]:
def replace_outlier_ages(group):
    if len(group) > 1:
        mode_age = group.mode()[0]
        group = group.apply(lambda x: x if x == mode_age else np.nan)
    return group

df['Age'] = df.groupby('Customer_ID')['Age'].transform(replace_outlier_ages)

FillMissingWithGroupMode(df, 'Customer_ID', 'Age')

In [ ]:
def replace_outlier_loan(group):
    if len(group) > 1:
        mode_age = group.mode()[0]
        group = group.apply(lambda x: x if x == mode_age else np.nan)
    return group

df['Num_of_Loan'] = df.groupby('Customer_ID')['Num_of_Loan'].transform(replace_outlier_loan)

FillMissingWithGroupMode(df, 'Customer_ID', 'Num_of_Loan')


In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
missing_values = df.isnull().sum()
missing_values[missing_values > 0] #<--- missing values are from test dataset which we merged with train dataset
# I realized that the Credit_Score column values from test set were NaN, so I will split the data into train and test sets based on the Credit_Score column

**2. Exploratory Data Analysis (EDA) and Handling Extreme Outliers and error**

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame

def detect_outliers(dataframe):
    outlier_indices_dict = {}

    # Loop over each column in the DataFrame
    for column in dataframe.select_dtypes(include=[np.number]).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = dataframe[column].quantile(0.05)
        Q3 = dataframe[column].quantile(0.95)
        IQR = Q3 - Q1

        # Define bounds for outliers
        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)

        # Find outliers
        outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
        outlier_indices_dict[column] = outliers.index.tolist()

    return outlier_indices_dict

# Run the function to detect outliers in all numerical columns
outliers_dict = detect_outliers(df)

# Example to print the outliers for a column
for column, indices in outliers_dict.items():
    print(f"Outliers in column {column}: {len(indices)}")

In [ ]:
# Pre-calculate the typical income for each customer
typical_incomes = df.groupby('Customer_ID')['Annual_Income'].median()

# Merge the median income back to the original dataframe
df = df.join(typical_incomes.rename('Median_Income'), on='Customer_ID')

# Calculate the threshold for being considered an outlier
threshold = 0.5  # You can adjust this threshold as needed
df['Income_Lower_Threshold'] = df['Median_Income'] * (1 - threshold)
df['Income_Upper_Threshold'] = df['Median_Income'] * (1 + threshold)

# Replace outliers with the median income
outlier_condition = (
    (df['Annual_Income'] < df['Income_Lower_Threshold']) |
    (df['Annual_Income'] > df['Income_Upper_Threshold'])
)
df.loc[outlier_condition, 'Annual_Income'] = df.loc[outlier_condition, 'Median_Income']

# Drop the extra columns if you no longer need them
df.drop(['Median_Income', 'Income_Lower_Threshold', 'Income_Upper_Threshold'], axis=1, inplace=True)


In [ ]:
upper_limit = df['Annual_Income'].quantile(0.95)  # Extreme outliers

df['Annual_Income'] = df['Annual_Income'].apply(lambda x: min(x, upper_limit))

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(df['Annual_Income'], bins=50, color='blue', edgecolor='black')  # Reduced number of bins for clarity
plt.title('Capped Distribution of Annual Income')
plt.xlabel('Annual Income')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 3))
plt.hist(df['Age'], bins=60, color='blue', edgecolor='black')
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the upper and lower limits
upper_limit = df['Num_Bank_Accounts'].quantile(0.95)  # For extreme outliers
lower_limit = 0  # Minimum realistic value

df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x: min(max(x, lower_limit), upper_limit))

plt.figure(figsize=(9, 3))
plt.hist(df['Num_Bank_Accounts'], bins=40, color='blue', edgecolor='black')
plt.title('Distribution of Num_Bank_Accounts')
plt.xlabel('Num_Bank_Accounts')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define the upper and lower limits
upper_limit = df['Num_Credit_Card'].quantile(0.95)  # For extreme outliers
lower_limit = 0  # Minimum realistic value

df['Num_Credit_Card'] = df['Num_Credit_Card'].apply(lambda x: min(max(x, lower_limit), upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Num_Credit_Card'], bins=40, color='blue', edgecolor='black')
plt.title('Distribution of Num_Credit_Card')
plt.xlabel('Num_Credit_Card')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_limit = df['Interest_Rate'].quantile(0.95)  # Extreme outliers

df['Interest_Rate'] = df['Interest_Rate'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Interest_Rate'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Interest_Rate')
plt.xlabel('Interest_Rate')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_limit = df['Total_EMI_per_month'].quantile(0.95)  # Extreme outliers

df['Total_EMI_per_month'] = df['Total_EMI_per_month'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Total_EMI_per_month'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Total_EMI_per_month')
plt.xlabel('Total_EMI_per_month')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_limit = df['Amount_invested_monthly'].quantile(0.95)  # Extreme outliers

df['Amount_invested_monthly'] = df['Amount_invested_monthly'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Amount_invested_monthly'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Amount_invested_monthly')
plt.xlabel('Amount_invested_monthly')
plt.ylabel('Frequency')
plt.show()

**3. Feature Engineering:**

In [ ]:
categorical_columns =df.select_dtypes('O').columns
display(categorical_columns)

In [ ]:
categorical_columns_head = df.select_dtypes('O').head(5)
categorical_columns_head

In [ ]:
for col in categorical_columns:
    print(f"\nColumn: {col}")
    print("Unique values count:", df[col].nunique())
    print("Value counts:")
    #print(df[col].value_counts(dropna=False).head(1))

In [ ]:
df.drop('Name', axis=1, inplace=True) #<--- dropping 'Name' column as it has too many unique values
df = pd.get_dummies(df, columns=['Occupation', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour'], drop_first=True)

df.columns


In [ ]:
print(df['Type_of_Loan'].value_counts(dropna=False).head(7))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

df['Type_of_Loan'] = df['Type_of_Loan'].str.split(',')

mlb = MultiLabelBinarizer()

type_of_loan_mlb = pd.DataFrame(mlb.fit_transform(df['Type_of_Loan']), columns=mlb.classes_, index=df.index)
df = df.join(type_of_loan_mlb)

df.drop('Type_of_Loan', axis=1, inplace=True)

df.head()


In [ ]:
numerical_columns = df.select_dtypes('number').columns
display(numerical_columns)

In [ ]:
numerical_columns_head = df.select_dtypes('number').head(10)
numerical_columns_head

In [ ]:
agg_functions = ['mean', 'median', 'max', 'min', 'std']
aggregated_df = df.groupby('Customer_ID').agg({
    'Annual_Income': agg_functions,
    'Num_of_Loan': agg_functions,
    'Outstanding_Debt': agg_functions,
}).reset_index()

# Rename columns except for 'Customer_ID'
aggregated_df.columns = ['Customer_ID'] + ['{}_{}'.format(col[0], col[1]) for col in aggregated_df.columns[1:]]

# Merge the DataFrames
df = df.merge(aggregated_df, on='Customer_ID', how='left')
df.head()

In [ ]:
# Creating a new feature as the ratio of Outstanding_Debt to Annual_Income
df['Debt_Income_Ratio'] = df['Outstanding_Debt'] / df['Annual_Income']

# Calculate the ratio of Monthly_Balance to Monthly_Inhand_Salary
df['Balance_Salary_Ratio'] = df['Monthly_Balance'] / df['Monthly_Inhand_Salary']

# Calculate the ratio of Total_EMI_per_month to Monthly_Inhand_Salary
df['EMI_Salary_Ratio'] = df['Total_EMI_per_month'] / df['Monthly_Inhand_Salary']

# Calculate the ratio of Outstanding_Debt to Annual_Income
df['Debt_Income_Ratio'] = df['Outstanding_Debt'] / df['Annual_Income']

df.isna().sum()

In [ ]:
def credit_utilization_category(ratio):
    if ratio < 30:
        return 'Low'
    elif ratio < 60:
        return 'Medium'
    else:
        return 'High'

# Apply the function to create a new categorical feature
df['Credit_Utilization_Category'] = df['Credit_Utilization_Ratio'].apply(credit_utilization_category)

# One-hot encoding of the new categorical feature
df = pd.get_dummies(df, columns=['Credit_Utilization_Category'], drop_first=True)



**4. Splitting the data: reminder that original test.csv target variable was already dropped**

from sklearn.model_selection import train_test_split
df = df.drop(columns=['Customer_ID'], errors='ignore') #<--- dropping 'Customer_ID' column as it is not needed for training
#<--- Splitting the data into train and test sets
train_set = df[df['Credit_Score'].notna()]
test_set = df[df['Credit_Score'].isna()]

#<--- remove the target variable from the test set (it is already missing, so just for precaution)
test_set = test_set.drop(columns=['Credit_Score'], errors='ignore')

# Splitting the train_set further into training and validation sets
X = train_set.drop('Credit_Score', axis=1)  # Features
y = train_set['Credit_Score']  # Target variable

#<--- for performance review (train_set)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

#<--- for final predictions (test_set)
X_test = test_set


#<--- This is most because We initially had Trainv.csv and Testv.csv, but we merged them together to do the data cleaning and feature engineering consistently on both datasets. Now, we will split them back into train and test sets.
#<--- but part of our data has missing Credit_Score missing. Based on that we can't measure the performance of our model. So, we will split the data into train and test sets.
#<--- So we will use train_set for training and validation sets and test_set for final predictions.


In [ ]:
from sklearn.model_selection import GroupKFold

train_df = df[df['Credit_Score'].notna()]   #<--- Splitting the data into train and test sets
test_df = df[df['Credit_Score'].isna()].drop(columns=['Credit_Score'], errors='ignore')

X = train_df.drop(columns=['Credit_Score'])
y = train_df['Credit_Score']
groups = train_df['Customer_ID']  # Groups for GroupKFold

#<--- i decided to use GroupKFold because the dataset has customers with multiple rows. So, I want to make sure that the same customer doesn't appear in both training and validation sets. This is to avoid data leakage and overfitting.
gkf = GroupKFold(n_splits=5)

train_idx, valid_idx = next(gkf.split(X, y, groups=groups))
X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

#<--- now we can drop the Customer_ID column as it is not needed for training anymore
X_train = X_train.drop(columns=['Customer_ID'])
X_valid = X_valid.drop(columns=['Customer_ID'])


#<--- for final predictions (test_set)
X_test = test_df


#<--- This is most because We initially had Trainv.csv and Testv.csv, but we merged them together to do the data cleaning and feature engineering consistently on both datasets. Now, we will split them back into train and test sets.
#<--- but part of our data has missing Credit_Score missing. Based on that we can't measure the performance of our model. So, we will split the data into train and test sets.
#<--- So we will use train_set for training and validation sets and test_set for final predictions.


In [ ]:
print("inf in X_train:", X_train.isin([np.inf, -np.inf]).sum().sum())

import numpy as np

# Identify columns with infinite values
inf_columns = X_train.columns[X_train.isin([np.inf, -np.inf]).any()].tolist()

print("Columns with Infinite Values:", inf_columns)



In [ ]:

print("extreme values in X_train:", X_train.abs().max())

**5. Feature Selection: Correlation, RandomForest Importance, RFECV**

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

#These codes run for so long that, I preemvlively chosen RF and LightGBM as my models so that I can skip scaling.
#I could have also engineered some features but I already have too many features.

##<--- 1. Correlation Analysis on Training Data
corr_matrix = X_train.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.xticks(rotation=45, ha='right')  # rotate for better readability
plt.yticks(rotation=0)  # <--- for better readability
plt.title("Correlation Matrix for Training Data")
plt.tight_layout()  # <--- to fit
plt.show()

#<--- identify highly correlated variables (0.85)
high_corr_var = [col for col in corr_matrix.columns if any(corr_matrix[col] > 0.85)]
print("Highly correlated variables:", high_corr_var)

##<--- 2. Feature Importance with Random Forest on Training Data
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
feature_importances = pd.DataFrame(rf.feature_importances_, index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

#<--- plot the feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x=feature_importances['importance'], y=feature_importances.index)
plt.title("Feature Importances in Training Data")
plt.show()

##<--- 3. Recursive Feature Elimination with Cross-Validation (RFECV) on Training Data
rfecv = RFECV(estimator=rf, step=5, cv=StratifiedKFold(5), scoring='accuracy', n_jobs=-1)
rfecv.fit(X_train, y_train)

print("Optimal number of features:", rfecv.n_features_)

#<--- extracting the mean cross-validation scores
mean_scores = rfecv.cv_results_['mean_test_score']

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(mean_scores) + 1), mean_scores, marker='o')
plt.xlabel("Number of features selected")
plt.ylabel("Cross-validation score (nb of correct classifications)")
plt.title("RFECV - Number of Features vs. CV Score")
plt.show()

#<--- storting final selected features
selected_features = X_train.columns[rfecv.support_]
print("Selected Features:", selected_features)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Assume X_train, y_train, and groups are already defined

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)

# Initialize the list to store feature importances and accuracy scores
feature_importances_list = []
scores = []

# GroupKFold for splitting training data to avoid leakage
gkf = GroupKFold(n_splits=5)

for train_index, test_index in gkf.split(X_train, y_train, groups=groups):
    # Split the data while keeping groups together
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Fit the model on the training fold
    rf.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold and calculate accuracy
    y_valid_pred_fold = rf.predict(X_valid_fold)
    fold_score = accuracy_score(y_valid_fold, y_valid_pred_fold)
    scores.append(fold_score)

    # Store the feature importances
    feature_importances_list.append(rf.feature_importances_)

# Calculate the average feature importances across all folds
average_feature_importances = np.mean(feature_importances_list, axis=0)

# Create a DataFrame for easy plotting and manipulation
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': average_feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(12, 10))
sns.barplot(data=feature_importances_df.head(30), x='Importance', y='Feature')  # Plot top 30 for visibility
plt.title("Feature Importances in Training Data")
plt.show()

# Print the average accuracy across all folds
print("Average accuracy across folds:", np.mean(scores))

# Select the top N features by importance
N = 30  # or another number based on your analysis
selected_features = feature_importances_df.head(N)['Feature']
print("Selected Features:", selected_features)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, y_train, X_valid, and y_valid are already defined
# Assuming selected_features is the list of selected features after feature selection

# Training Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)

# Fit the model on the training data using only the selected features
rf_classifier.fit(X_train[selected_features], y_train)

# Predict on the training data
y_train_pred = rf_classifier.predict(X_train[selected_features])

# Calculate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Set Accuracy:", train_accuracy)

# Predict on the validation data
y_valid_pred = rf_classifier.predict(X_valid[selected_features])

# Calculate accuracy on the validation set
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Set Accuracy:", valid_accuracy)

# Optionally, print a classification report for detailed performance metrics on the validation set
print(classification_report(y_valid, y_valid_pred))


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_valid, y_valid_pred))

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np

# Assuming rf_classifier is the trained Random Forest model
# and X_train[selected_features], y_train are the training data and labels

# Generate learning curves
train_sizes, train_scores, valid_scores = learning_curve(
    estimator=rf_classifier,
    X=X_train[selected_features],
    y=y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

# Calculate mean and standard deviation for training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for validation set scores
valid_mean = np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.15)

plt.plot(train_sizes, valid_mean, label='Cross-validation score', color='green', marker='o')
plt.fill_between(train_sizes, valid_mean - valid_std, valid_mean + valid_std, color='green', alpha=0.15)

plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()


In [ ]:
X_test_selected = X_test[selected_features]
y_test_pred = rf_classifier.predict(X_test_selected)